# Modelado de churn de Telecom X


## Carga de datos
Se importan librerías principales y se carga el dataset tratado.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

sns.set(style='whitegrid')
pd.set_option('display.max_columns', None)

df = pd.read_csv('data/telecom_clientes_tratado.csv')
df['Charges.Total'].fillna(df['Charges.Total'].mean(), inplace=True)
df.head()


## Exploración y limpieza
Revisión de valores faltantes y eliminación de columnas irrelevantes.


In [ ]:
# Eliminación de identificadores y exploración inicial
df.drop(columns=['customerID'], inplace=True, errors='ignore')
churn_rate = df['Churn'].value_counts(normalize=True)
print('Distribución porcentual de churn:')
print(churn_rate)
sns.countplot(x='Churn', data=df)
plt.title('Distribución de clientes con y sin churn')
plt.show()


## Codificación y preparación de variables
Transformación de variables categóricas y separación de variables predictoras.


In [ ]:
df_encoded = pd.get_dummies(df, drop_first=True)
X = df_encoded.drop('Churn_Yes', axis=1)
y = df_encoded['Churn_Yes']
df_encoded.head()


## Análisis de correlación y visualizaciones
Relaciones entre variables y la variable objetivo.


In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_encoded.corr(), cmap='coolwarm', center=0)
plt.title('Matriz de correlación')
plt.show()
cor_target = df_encoded.corr()['Churn_Yes'].sort_values(ascending=False)
print(cor_target)


In [ ]:
sns.boxplot(data=df, x='Churn', y='tenure')
plt.title('Tenure vs Churn')
plt.show()
sns.boxplot(data=df, x='Churn', y='Charges.Total')
plt.title('Gasto Total vs Churn')
plt.show()


## División de datos y escalado
Separación en conjuntos de entrenamiento y prueba y escalado de variables.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Balanceo de clases con SMOTE
Se aplica SMOTE para equilibrar la proporción de clases antes del entrenamiento.


In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
scaler_smote = StandardScaler()
X_train_smote_scaled = scaler_smote.fit_transform(X_train_smote)
X_test_smote_scaled = scaler_smote.transform(X_test)


## Entrenamiento de modelos (Logistic Regression, Random Forest, XGBoost)
Entrenamiento de diferentes algoritmos para predecir la cancelación.


In [ ]:
# Regresión Logística
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train_smote_scaled, y_train_smote)

# Random Forest con búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
rf = RandomForestClassifier(random_state=42)
rf_random = RandomizedSearchCV(rf, param_dist, n_iter=10, cv=3, scoring='f1', random_state=42, n_jobs=-1)
rf_random.fit(X_train_smote, y_train_smote)
rf_best = rf_random.best_estimator_

# XGBoost
xgb = XGBClassifier(random_state=42, eval_metric='logloss')
xgb.fit(X_train_smote, y_train_smote)


## Evaluación de modelos y ajuste de umbrales
Medición de desempeño y ajuste del umbral para Regresión Logística.


In [ ]:
# Predicciones y evaluación con umbral estándar
print('📊 Logistic Regression (umbral=0.50):')
print(classification_report(y_test, lr.predict(X_test_smote_scaled)))
print(confusion_matrix(y_test, lr.predict(X_test_smote_scaled)))

print('📊 Random Forest:')
print(classification_report(y_test, rf_best.predict(X_test)))
print(confusion_matrix(y_test, rf_best.predict(X_test)))

print('📊 XGBoost:')
print(classification_report(y_test, xgb.predict(X_test)))
print(confusion_matrix(y_test, xgb.predict(X_test)))

# Ajuste de umbral para Regresión Logística
umbral_optimo = 0.35
y_probs = lr.predict_proba(X_test_smote_scaled)[:, 1]
y_pred_umbral = (y_probs >= umbral_optimo).astype(int)
print(f'
Evaluación Logistic Regression con umbral {umbral_optimo}:')
print(classification_report(y_test, y_pred_umbral))
print(confusion_matrix(y_test, y_pred_umbral))


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = roc_auc_score(y_test, y_probs)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - Regresión Logística')
plt.legend(loc='lower right')
plt.show()


## Importancia de variables
Identificación de las variables más relevantes en los modelos.


In [ ]:
lr_importance = pd.Series(lr.coef_[0], index=X.columns).sort_values(ascending=False)
print('Importancia (coeficientes) - Logistic Regression:')
print(lr_importance.head(10))


In [ ]:
rf_importance = pd.Series(rf_best.feature_importances_, index=X.columns).sort_values(ascending=False)
print('Importancia - Random Forest:')
print(rf_importance.head(10))


## Comparación y resultados
Resumen de métricas para cada modelo entrenado.


In [ ]:
import pandas as pd
def metricas(y_true, pred):
    report = classification_report(y_true, pred, output_dict=True)
    return [report['accuracy'], report['weighted avg']['precision'], report['weighted avg']['recall'], report['weighted avg']['f1-score']]

resultados = pd.DataFrame(
    index=['Logistic Regression', 'Random Forest', 'XGBoost'],
    columns=['Accuracy', 'Precision', 'Recall', 'F1']
)
resultados.loc['Logistic Regression'] = metricas(y_test, y_pred_umbral)
resultados.loc['Random Forest'] = metricas(y_test, rf_best.predict(X_test))
resultados.loc['XGBoost'] = metricas(y_test, xgb.predict(X_test))
print(resultados)


## Conclusión estratégica
- **Variables influyentes:** permanencia del cliente (tenure), contratos mes a mes, falta de soporte técnico y cargos totales elevados.
- **Mejor modelo:** XGBoost presentó el mayor desempeño global frente a Random Forest y Regresión Logística.
- **Perfil de cliente propenso a cancelar:** usuarios de corta permanencia con servicios mensuales y pagos mediante *electronic check*.
- **Acciones de retención:** ofrecer descuentos por contratos anuales, reforzar soporte técnico y campañas proactivas para clientes con altos cargos mensuales.
